In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display
from openai import OpenAI
import locale
import numpy as np
from sklearn.metrics import accuracy_score
import utils.extractors as extractors
import utils.millers as millers
import gradio as gr
from datetime import datetime
from functools import partial
from sqlalchemy import text

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Load environment variables from .env
load_dotenv()

# Data Extraction

## Categories

In [ ]:
df_cat = extractors.fetch_categories()
df_cat

## Transactions

In [ ]:
df_trans = extractors.fetch_sample_transactions(source='sapphire_reserve', limit=1_000)
df_trans.loc[:, 'category_name'] = None
df_trans

# Dashboard

In [ ]:
def category_change(trans_id, category_name):
    df_trans.loc[trans_id, 'category_name'] = category_name

def fetch_offset_transaction(offset, trans_id):
    new_idx = df_trans.index.get_loc(trans_id) + offset

    if new_idx < 0 or new_idx >= len(df_trans):
        raise gr.Error('No more of those crazy transactions!')

    new_transaction = df_trans.iloc[new_idx]

    trans_id = new_transaction.name
    trans_date = datetime.combine(new_transaction['transaction_date'], datetime.min.time())
    trans_desc = new_transaction['transaction_description']
    trans_amount = new_transaction['amount']
    category_name = new_transaction['category_name']

    return trans_id, trans_date, trans_desc, trans_amount, category_name

fetch_prev_transaction = partial(fetch_offset_transaction, -1)
fetch_next_transaction = partial(fetch_offset_transaction,  1)

transaction = df_trans.iloc[0]

with gr.Blocks(theme='Glass') as demo:
    with gr.Row():
        trans_id = gr.Textbox(
            value=transaction.name,
            interactive=False,
            label='ID'
        )
        trans_date = gr.DateTime(
            value=datetime.combine(transaction['transaction_date'], datetime.min.time()),
            interactive=False,
            include_time=False,
            label='Transaction Date'
        )
    with gr.Row():
        trans_desc = gr.Textbox(
            value=transaction['transaction_description'],
            interactive=False,
            label='Description'
        )
        trans_amount = gr.Number(
            value=transaction['amount'],
            interactive=False,
            label='Amount'
        )

    with gr.Row():
        category_name = gr.Radio(
            choices=df_cat.category_name.unique().tolist(),
            interactive=True,
            label='Category'
        )

    category_name.change(
        category_change,
        inputs=[trans_id, category_name],
        outputs=None
    )

    with gr.Row():
        iter_btn_outputs = [trans_id, trans_date, trans_desc, trans_amount, category_name]
        
        previous_btn = gr.Button('Previous')
        previous_btn.click(fn=fetch_prev_transaction, inputs=trans_id, outputs=iter_btn_outputs)

        next_btn = gr.Button('Next')
        next_btn.click(fn=fetch_next_transaction, inputs=trans_id, outputs=iter_btn_outputs)

demo.launch()

In [ ]:
df_labeled = (df_trans.query('category_name==category_name')
              .reset_index()
              .merge(df_cat.reset_index().loc[:, ['category_name', 'category_id']], how='left', on='category_name')
              .set_index('transaction_id'))
df_labeled.shape

In [ ]:
upsert_text = """\
INSERT INTO sources.transaction_categories (transaction_id, category_id)
VALUES (:transaction_id, :value)
ON CONFLICT (transaction_id) DO UPDATE
SET transaction_id = EXCLUDED.transaction_id, category_id = EXCLUDED.category_id;
"""
with extractors.engine.begin() as conn:
    for idx, row in df_labeled.iterrows():
        conn.execute(
            text(upsert_text),
            {"value": int(row['category_id']), "transaction_id": idx}
        )